L'esame sarà valutato

- verificando la correttezza dell'output generato dalle funzioni nel notebook
- analizzando il codice sottomesso

L'esame è composto dai 4 esercizi descritti in calce. Al completamento dei 4 esercizi,

- Modificala la variabile `NOME_COGNOME` indicata nella cella sotto
- esporta il tuo notebook tramite File - Export - IPython Notebook
- caricalo su [questo indirizzo](https://docs.google.com/forms/d/e/1FAIpQLSdH9DBaPyYIcDOtjZiMmgXFhtfQ-UXDEs7X1RiLzx7S6q-8LA/viewform?usp=sf_link) entro la fine del tempo prestabilito ⏳

In [0]:
# sostituisci con il tuo nome e cognome
NOME_COGNOME = "bill_gates"

## Dataset

In this exam, we'll consider the San Francisco Fire Department Calls dataset.

```
/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv
```

## Part 1

Write a function named `prepare_dataframe()` that

- reads the csv file
- excludes the records with `CallType` that occurs less than `200` times in the entire dataset (it is fine to hardcode the values manually)
- computes a new column `CallDayOfWeek` that computes the day of the week (1 Sunday, 7 Saturday) based on the column `CallDate`
- computes a new column `DeltaPriority` that computes the difference between `FinalPriority` and `OriginalPriority` (eg `DeltaPriority=2` when `FinalPriority=4` and `OriginalPriority=2`)
- keeps only the following columns `CallNumber`, `CallType`, `Priority`, `CallDayOfWeek`, `DeltaPriority`, `Delay`
- remove rows where any record is null (you can use `dropna`)

In [0]:
def prepare_dataframe() -> DataFrame:
    # your code here
    return df

In [0]:
# you can test it by
display(
    prepared_dataframe()
)

## Part 2

Write a function named `define_pipeline()` that

- returns a `Pipeline` object
- the object is initialized with stages that predict the `Delay` via a linear regression based on the following features: `CallType`, `Priority`, `CallDayOfWeek`, `DeltaPriority`. Two of these features can be considered as categorical, which are the 2?

In [0]:
from pyspark.ml import Pipeline

def define_pipeline() -> Pipeline:
    # your code here
    return pipeline

## Part 3

Write a function named `fit_model(df, pipeline)` that
- takes as input
  - `df`, a `DataFrame` object that corresponds to the one that is returned by `prepare_dataframe()`
  - `pipeline`, a `Pipeline` object that corresonds to the one that is returned by `define_pipeline()`
- splits `df` in _train_ and _test_ dataset (80% and 20% respectively using a seed of `42`)
- fits the `pipeline` on the _train_ dataset and runs prediction on the _test_ dataset
- returns the `DataFrame` of the predictions made of only 2 columns: `DelayLabel`, `DelayPrediction`

In [0]:
def fit_model(df: DataFrame, pipeline: Pipeline) -> DataFrame:
    # your code here
    return predictions_df

## Part 4
Write a function `evaluate_r2(predictions_df)` that
- takes as input a `DataFrame` object that corresponds to the one that is returned by `fit_model()`, ie made of 2 columns `DelayLabel` and `DelayPrediction`
- computes the R squared metric and returns the R2 value

In [0]:
def evaluate_r2(predictions_df: DataFrame) -> float:
    # your code here
    return r2

## Solution

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType
from pyspark.sql.functions import to_date, dayofweek

def prepare_dataframe():
    sf_fire_file = "/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"

    fire_schema = StructType(
        [
            StructField('CallNumber', IntegerType(), True),
            StructField('UnitID', StringType(), True),
            StructField('IncidentNumber', IntegerType(), True),
            StructField('CallType', StringType(), True),                  
            StructField('CallDate', StringType(), True),      
            StructField('WatchDate', StringType(), True),
            StructField('CallFinalDisposition', StringType(), True),
            StructField('AvailableDtTm', StringType(), True),
            StructField('Address', StringType(), True),       
            StructField('City', StringType(), True),       
            StructField('Zipcode', IntegerType(), True),       
            StructField('Battalion', StringType(), True),                 
            StructField('StationArea', StringType(), True),       
            StructField('Box', StringType(), True),       
            StructField('OriginalPriority', IntegerType(), True),       
            StructField('Priority', IntegerType(), True),       
            StructField('FinalPriority', IntegerType(), True),       
            StructField('ALSUnit', BooleanType(), True),       
            StructField('CallTypeGroup', StringType(), True),
            StructField('NumAlarms', IntegerType(), True),
            StructField('UnitType', StringType(), True),
            StructField('UnitSequenceInCallDispatch', IntegerType(), True),
            StructField('FirePreventionDistrict', StringType(), True),
            StructField('SupervisorDistrict', StringType(), True),
            StructField('Neighborhood', StringType(), True),
            StructField('Location', StringType(), True),
            StructField('RowID', StringType(), True),
            StructField('Delay', FloatType(), True)
        ]
    )

    df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)
    return df\
            .where(~df.CallType.isin(['Administrative', 'Train / Rail Fire', 'Lightning Strike (Investigation)']))\
            .withColumn(
                "CallDayOfWeek",
                dayofweek(to_date(df.CallDate, "MM/dd/yyyy"))
            )\
            .withColumn('DeltaPriority', df.FinalPriority - df.OriginalPriority)\
            .select('CallNumber', 'CallType', 'Priority', 'CallDayOfWeek', 'DeltaPriority', 'Delay')\
            .dropna(how='any')

In [0]:
# you can test it by
df = prepare_dataframe()
display(
    prepare_dataframe()
)

CallNumber,CallType,Priority,CallDayOfWeek,DeltaPriority,Delay
20110014,Medical Incident,1,6,1,5.233333
20110015,Medical Incident,1,6,1,3.0833333
20110016,Structure Fire,3,6,0,3.05
20110016,Structure Fire,3,6,0,2.3166666
20110016,Structure Fire,3,6,0,3.0166667
20110016,Structure Fire,3,6,0,2.6833334
20110016,Structure Fire,3,6,0,2.1
20110016,Structure Fire,3,6,0,2.7166667
20110016,Structure Fire,3,6,0,2.7666667
20110016,Structure Fire,3,6,0,3.2666667


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression

def define_pipeline() -> Pipeline:
    indexer = StringIndexer(
        inputCols=["CallType", "CallDayOfWeek"],
        outputCols=["CallTypeIndex", "CallDayOfWeekIndex"],
        handleInvalid='skip'
    )
    encoder = OneHotEncoder(
        inputCols=["CallTypeIndex", "CallDayOfWeekIndex"],
        outputCols=["CallTypeOhe", "CallDayOfWeekOhe"]
    )
    assembler = VectorAssembler(
        inputCols=["CallTypeOhe", "Priority", "CallDayOfWeekOhe", "DeltaPriority"],
        outputCol="features"
    )
    regressor = LinearRegression(featuresCol="features", labelCol="Delay")
    return Pipeline(
        stages=[indexer, encoder, assembler, regressor]
    )

In [0]:
pipeline = define_pipeline()

In [0]:
from pyspark.sql import DataFrame

def fit_model(df: DataFrame, pipeline: Pipeline) -> DataFrame:
    train, test = df.randomSplit([.8, .2], seed=42)
    model = pipeline.fit(train)
    predictions = model.transform(test)
    return predictions\
            .withColumnRenamed('Delay', 'DelayLabel')\
            .withColumnRenamed('prediction', 'DelayPrediction')\
            .select('DelayLabel', 'DelayPrediction')

In [0]:
predictions = fit_model(df, pipeline)
display(fit_model(df, pipeline))

DelayLabel,DelayPrediction
2.1,3.639485181029201
2.7666667,3.639485181029201
3.0166667,3.639485181029201
6.116667,6.081963305033955
2.4333334,3.5856765369974415
3.8666666,3.5856765369974415
3.6666667,3.5856765369974415
2.2833333,4.276048219875058
2.7833333,3.5856765369974415
3.5166667,3.5856765369974415


In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

def evaluate_r2(predictions_df: DataFrame) -> float:
    evaluator = RegressionEvaluator(
        predictionCol="DelayPrediction",
        labelCol="DelayLabel",
        metricName="r2"
    )

    return evaluator.evaluate(predictions)

In [0]:
print(
        "R2 ", evaluate_r2(predictions)
)

R2  0.005794662725845279
